In [3]:
# Import required mudules

import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from itertools import combinations

In [4]:
# Read in the csv file as a DataFrame

sap = pd.read_csv("sphist.csv", index_col=False)

# Convert the Date column to datetime

sap["Date"] = pd.to_datetime(sap["Date"])

# The dataframe is currently sored by date in descending order
# Let's change it to ascending order
# Also reset index so starts at 0

sap = sap.sort_values(by="Date", ascending=True)

sap = sap.reset_index(drop=True)

In [5]:
# Calculate 5 days moving average, 30 day moving average
# 5 day standard deviation and 30 day standerd deviation and and them as columns

# ma_5
def add_moving_average_col(period):
    control = []
    col = []
    for row in sap.iterrows():
        row = row[1]
        if len(control) < period:
            control.append(row["Close"])
            col.append(0)
        else:
            col.append(np.mean(control))
            del control[0]
            control.append(row["Close"])
    sap[f"{period}_day_ma"] = col

add_moving_average_col(5)

# ma_30
add_moving_average_col(30)

# std_5
def add_moving_std_col(period):
    control = []
    col = []
    for row in sap.iterrows():
        row = row[1]
        if len(control) < period:
            control.append(row["Close"])
            col.append(0)
        else:
            col.append(np.std(control))
            del control[0]
            control.append(row["Close"])
    sap[f"{period}_day_std"] = col

add_moving_std_col(5)

# std_30
add_moving_std_col(30)

In [7]:
# Also add the prior day close and volume as columns

# pr_day_close
def prior_day(attribute):
    col = sap[attribute].shift(1)
    sap[f"pr_day_{attribute}".lower()] = col

prior_day("Close")

# pr-day_volume
prior_day("Volume")

In [8]:
# Remove the first 30 rows as these rows do not have values for
# all indicators
sap.drop(range(0,31), inplace=True)

# Drop na rows
sap.dropna(axis=0, inplace=True)

In [9]:
# Split df into train and test sets with train being all datapoints
# before 2013-01-01
train = sap.loc[sap["Date"] < datetime(year=2013, month=1, day=1)]
test = sap.loc[sap["Date"] >= datetime(year=2013, month=1, day=1)]

In [47]:
# Let's build, train and test a model with absolute mean error
# as the error metric

# For features we can only use metrics we would know at the end
# of the previous day
all_features = ['5_day_ma', '30_day_ma', '5_day_std', '30_day_std',
'pr_day_close', 'pr_day_volume']

target = "Close"

def train_and_test():

# In order to find the best combination of columns to use
# generate a model for every combination and calculate and
# compare error metrics

    maes = {}
    combinations_list = []

    for i in range(1, len(all_features)):

        new_combinations = combinations(all_features, i)

        for combination in new_combinations:

            combinations_as_a_list = list(combination)
            # In order to store the list as a dictionary value it is
            # necessary to join the items into a string and then
            # split them out to a list later
            key = '-'.join(combinations_as_a_list)
            combinations_list.append(key)

    for features in combinations_list:

        features = features.split('-')

        model = LinearRegression()
        model.fit(train[features], train[target])
        predictions = model.predict(test[features])

        mae = mean_absolute_error(test[target], predictions)

        features = '-'.join(features)

        maes[features] = mae

    # Turn the dict into a dataframe

    df = pd.DataFrame.from_dict(maes, orient="index")
    df.columns = ["mae"]
    
    # return the dataframe
    return df

df = train_and_test()

In [49]:
# sort the df by mae
df_sorted = df.sort_values(by="mae")

# Inspect the highest and lowest values
print(df_sorted.head(10))
print(df_sorted.tail(10))

                                                          mae
30_day_ma-30_day_std-pr_day_close                   10.995101
30_day_ma-pr_day_close                              10.995633
30_day_ma-pr_day_close-pr_day_volume                10.997554
30_day_ma-5_day_std-30_day_std-pr_day_close         10.997932
30_day_ma-5_day_std-pr_day_close                    10.998866
30_day_ma-30_day_std-pr_day_close-pr_day_volume     10.999435
30_day_ma-5_day_std-30_day_std-pr_day_close-pr_...  11.003879
30_day_ma-5_day_std-pr_day_close-pr_day_volume      11.004355
pr_day_close                                        11.011899
pr_day_close-pr_day_volume                          11.012055
                                            mae
30_day_ma-30_day_std                  31.332082
30_day_ma-pr_day_volume               31.692958
30_day_ma                             31.964290
30_day_std-pr_day_volume             732.865697
5_day_std-30_day_std-pr_day_volume   743.740686
pr_day_volume                 

In [50]:
# Find the most common attributes in the poorly performing models

outlier_models = df_sorted.loc[df_sorted["mae"] > 100].index

def split_into_attributes(index_values):

    attribute_list = []

    for model in index_values:
        for attribute in model.split("-"):
            attribute_list.append(attribute)
    return pd.Series(attribute_list).value_counts()

split_into_attributes(outlier_models)

pr_day_volume    4
5_day_std        4
30_day_std       4
dtype: int64

In [51]:
# Find the top attributes in the top 10 models

split_into_attributes(df_sorted[:10].index)

pr_day_close     10
30_day_ma         8
pr_day_volume     5
5_day_std         4
30_day_std        4
dtype: int64

Thus we can see that of the 12 attributes in the worst performing models:
- pr_day_volume, 5_day_std and 30_day_std all appear 4 times
- the models have on average 1.7 attributes
- the worst three models and the only ones with error values over 800 are 5_day_std, 30_day_std and 5_day_std-30_day_std

Thus it appears that 5_day_std, 30_day_std and 5_day_std-30_day_std are not suitable metrics for predicting the Close price of the S&P 500. This is most probably due to the non-linear nature of the metrics

For the 10 best performing models:
- pr_day_close appears in every one
- pr_day_close is the 9th best performing model
- 30_day_ma apears in the top 8 best performing models
- the best performing model is 30_day_ma-30_day_std-pr_day_close which seems strange and is most likely a coincidence due to the otherwise bad performance of the 30_day_std metric
- the second best performing model is 0_day_ma-pr_day_close 

Thus it appears that pr_day_close and 30_day_ma are the most suitable metrics and that the actual best performing, reliable model has a mae of 10.995633 rather than 10.995101